In [ ]:
from pathlib import Path

import pandas as pd
import nibabel as nib
import numpy as np
from nilearn.image import clean_img

import warnings
warnings.filterwarnings("ignore")

bolds = !ls /opt/animalfmritools/animalfmritools/data/MouseAD/bids/derivatives/bold_preproc/sub-*/ses-*/func/*desc-preproc_bold.nii.gz

def read_confounds(tsv: Path) -> np.ndarray:

    confound_df = pd.read_csv(tsv, sep='\t')
    
    confounds_cols = []
    for i in confound_df.columns:
        if i == 'csf' or i == 'white_matter':
            confound_cols.append(i)

    regressors = confound_df.loc[1:][confound_cols].values # derivatives regressor's first element includes n/a - avoid

    return regressors

for bold_ix, bold in enumerate(bolds):

    progress = f"[{str(bold_ix+1).zfill(6)}/{str(len(bolds)).zfill(6)}]"

    # Paths
    bold = Path(bold)
    confound_tsv = Path(
        str(bold).replace("space-template_desc-preproc_bold.nii.gz", "desc-confounds_timeseries.tsv")
    )
    bold_denoised = Path(
        str(bold).replace("space-template_desc-preproc_bold.nii.gz", "space-template_desc-denoised_bold.nii.gz")
    )

    if not bold.exists() or not confound_tsv.exists():
        print(f"{progress} Skipping {bold.stem}\nConfound[.tsv] or bold[.nii.gz] was not found.")
        continue

    if bold_denoised.exists():
        print(f"{progress} Skipping {bold.stem}\n{bold_denoised.stem} already processed.")
        continue

    print(f"{progress} Processing {bold.stem}")

    # Load regressors
    nuisance_regressors = read_confounds(confound_tsv)

    # Load bold data
    bold_img = nib.load(bold).slicer[:,:,:,1:]

    # Denoise
    TR = bold_img.header.get_zooms()[-1]
    bold_denoised_img = clean_img(
        bold_img,
        confounds = nuisance_regressors,
        detrend = False,
        standardize = True,
        t_r = TR
    )
    bold_denoised_data = bold_denoised_img.get_fdata()
    mean_data = bold_img.get_fdata().mean(axis=3)[:,:,:,np.newaxis]
    bold_denoised_data += mean_data
    bold_denoised_img = nib.Nifti1Image(
        bold_denoised_data, 
        affine = bold_denoised_img.affine, 
        header = bold_denoised_img.header,
    )

    # Save
    nib.save(bold_denoised_img, bold_denoised)